In [2]:
import numpy as np
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer,tokenizer_from_json
import pandas as pd
from nltk.corpus import stopwords
import re
import json
STOPWORDS = set(stopwords.words('english'))
import os

In [2]:
embedding_dim=100
oov_token = '<OOV>'
train_test_split = 0.8
max_len = 200
trunc_type = 'post'
padding_type = 'post'

In [3]:
df=pd.read_csv("../DBPEDIA_test.csv")

In [4]:
labels = df.get('l1').values.tolist()
articles = df.get('text').values.tolist()
for i in range(len(articles)):
    for word in STOPWORDS:
        token = ' '+word+' '
        articles[i] = articles[i].replace(token,' ')
        articles[i] = re.sub(r'\s+', ' ',articles[i])

In [5]:
tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(articles)
sequences = tokenizer.texts_to_sequences(articles)
article_sequences = pad_sequences(sequences,maxlen=max_len,truncating=trunc_type,padding=padding_type)
word_index = tokenizer.word_index
vocab_size = len(word_index)

split_index = int(len(article_sequences)*train_test_split)
train_sequences = np.array(article_sequences[0:split_index])
test_sequences = np.array(article_sequences[split_index:])

label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
label_sequences = label_tokenizer.texts_to_sequences(labels)
train_label = np.array(label_sequences[0:split_index])
test_label = np.array(label_sequences[split_index:])

In [6]:
tokenizer_json = tokenizer.to_json()
label_tokenizer_json = label_tokenizer.to_json()

In [7]:
embeddings_index = {};
with open('../../glove.6B.100d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

In [8]:
if not os.path.exists('tokenizer.json'):
    with open('tokenizer.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
if not os.path.exists('label_tokenizer_json.json'):
    with open('label_tokenizer_json.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(label_tokenizer_json, ensure_ascii=False))

In [9]:
np.save('train_sequences.npy',train_sequences)
np.save('train_label.npy',train_label)
np.save('test_sequences.npy',test_sequences)
np.save('test_label.npy',test_label)
np.save('embeddings_matrix.npy',embeddings_matrix)

In [10]:
print(train_sequences.shape)
print(train_label.shape)
print(test_sequences.shape)
print(test_label.shape)
print(vocab_size)
print(word_index['i'])

(48635, 200)
(48635, 1)
(12159, 200)
(12159, 1)
226516
147


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, LSTM, Embedding,Conv2D,Bidirectional,MaxPool2D,Reshape
from keras.models import load_model
from keras.preprocessing.text import Tokenizer,tokenizer_from_json
from keras.callbacks import ModelCheckpoint
import json
import os
import datetime as dt
from matplotlib import pyplot as plt

In [12]:
with open('tokenizer.json', 'r', encoding='utf-8') as f1:
    tokenizer_config = json.load(f1)
with open('label_tokenizer_json.json', 'r', encoding='utf-8') as f2:
    label_tokenizer_config = json.load(f2)

In [5]:
train_sequences= np.load('train_sequences.npy')
train_label= np.load('train_label.npy')
test_sequences = np.load('test_sequences.npy')
test_label= np.load('test_label.npy')
embeddings_matrix= np.load('embeddings_matrix.npy')

In [6]:
tokenizer = tokenizer_from_json(tokenizer_config)
label_tokenizer = tokenizer_from_json(label_tokenizer_config)

NameError: name 'tokenizer_config' is not defined

In [15]:
embedding_dim = 100
word_index = word_index
vocab_size = vocab_size
max_len = 200
rnn_units = 100

In [16]:
model = Sequential()
model.add(Embedding(vocab_size+1,embedding_dim,input_length=max_len,weights=[embeddings_matrix], trainable=False))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(rnn_units,return_sequences=True),merge_mode='sum'))
model.add(Dropout(0.2))
model.add(Reshape((-1,rnn_units,1)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(10,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          22651700  
                                                                 
 dropout (Dropout)           (None, 200, 100)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 200, 100)         160800    
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 200, 100)          0         
                                                                 
 reshape (Reshape)           (None, 200, 100, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 198, 98, 64)       640       
                                                        

In [17]:
epochs = 10
save_dir = './saved_model'
save_fname = os.path.join(save_dir, "情感2.h5")
callbacks = [ModelCheckpoint(filepath=save_fname, monitor='loss', save_best_only=True)]
model.fit(train_sequences,train_label,epochs=epochs,validation_data=(test_sequences,test_label),callbacks=callbacks)

Epoch 1/10
1520/1520 [==============================] - 724s 473ms/step - loss: 1.5630 - accuracy: 0.5165 - val_loss: 1.4191 - val_accuracy: 0.5139
Epoch 2/10
1520/1520 [==============================] - 702s 462ms/step - loss: 1.3353 - accuracy: 0.5253 - val_loss: 1.2694 - val_accuracy: 0.5581
Epoch 3/10
1520/1520 [==============================] - 730s 480ms/step - loss: 1.2093 - accuracy: 0.5682 - val_loss: 1.1202 - val_accuracy: 0.6058
Epoch 4/10
1520/1520 [==============================] - 666s 438ms/step - loss: 1.0818 - accuracy: 0.6153 - val_loss: 0.9810 - val_accuracy: 0.6803
Epoch 5/10
1520/1520 [==============================] - 684s 450ms/step - loss: 0.9658 - accuracy: 0.6774 - val_loss: 0.8671 - val_accuracy: 0.7321
Epoch 6/10
1520/1520 [==============================] - 758s 499ms/step - loss: 0.8676 - accuracy: 0.7227 - val_loss: 0.7705 - val_accuracy: 0.7641
Epoch 7/10
1520/1520 [==============================] - 632s 416ms/step - loss: 0.7854 - accuracy: 0.7504 - val_

In [9]:
epochs = 10
save_dir = './saved_model'
save_fname = os.path.join(save_dir, "情感2.h5")
callbacks = [ModelCheckpoint(filepath=save_fname, monitor='loss', save_best_only=True)]
model.fit(train_sequences,train_label,epochs=epochs,validation_data=(test_sequences,test_label),callbacks=callbacks)

Epoch 1/10
1520/1520 [==============================] - 660s 434ms/step - loss: 0.5859 - accuracy: 0.8125 - val_loss: 0.5143 - val_accuracy: 0.8419
Epoch 2/10
1520/1520 [==============================] - 603s 396ms/step - loss: 0.5556 - accuracy: 0.8219 - val_loss: 0.4875 - val_accuracy: 0.8500
Epoch 3/10
1520/1520 [==============================] - 638s 420ms/step - loss: 0.5273 - accuracy: 0.8310 - val_loss: 0.4604 - val_accuracy: 0.8581
Epoch 4/10
1520/1520 [==============================] - 620s 408ms/step - loss: 0.5023 - accuracy: 0.8390 - val_loss: 0.4383 - val_accuracy: 0.8636
Epoch 5/10
1520/1520 [==============================] - 475s 312ms/step - loss: 0.4816 - accuracy: 0.8439 - val_loss: 0.4196 - val_accuracy: 0.8698
Epoch 6/10
1520/1520 [==============================] - 481s 317ms/step - loss: 0.4618 - accuracy: 0.8517 - val_loss: 0.4001 - val_accuracy: 0.8756
Epoch 7/10
1520/1520 [==============================] - 637s 419ms/step - loss: 0.4449 - accuracy: 0.8558 - val_

In [11]:
model.fit(train_sequences,train_label,epochs=epochs,validation_data=(test_sequences,test_label),callbacks=callbacks)

Epoch 1/10
1520/1520 [==============================] - 673s 443ms/step - loss: 0.3880 - accuracy: 0.8713 - val_loss: 0.3340 - val_accuracy: 0.8925
Epoch 2/10
1520/1520 [==============================] - 620s 408ms/step - loss: 0.3764 - accuracy: 0.8754 - val_loss: 0.3240 - val_accuracy: 0.8956
Epoch 3/10
1520/1520 [==============================] - 491s 323ms/step - loss: 0.3687 - accuracy: 0.8778 - val_loss: 0.3126 - val_accuracy: 0.9003
Epoch 4/10
1520/1520 [==============================] - 488s 321ms/step - loss: 0.3585 - accuracy: 0.8794 - val_loss: 0.3036 - val_accuracy: 0.9034
Epoch 5/10
1520/1520 [==============================] - 496s 326ms/step - loss: 0.3504 - accuracy: 0.8834 - val_loss: 0.2962 - val_accuracy: 0.9057
Epoch 6/10
1520/1520 [==============================] - 494s 325ms/step - loss: 0.3406 - accuracy: 0.8871 - val_loss: 0.2874 - val_accuracy: 0.9082
Epoch 7/10
1520/1520 [==============================] - 488s 321ms/step - loss: 0.3314 - accuracy: 0.8904 - val_

In [19]:
history = model.history
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss");

KeyError: 'accuracy'

In [1]:
from keras.models import load_model
import numpy as np
import json
from keras.preprocessing.text import Tokenizer, tokenizer_from_json

model = load_model('./saved_model/情感2.h5')

In [12]:
import joblib
joblib.dump(model,"主题分析.pkl")
joblib.dump(history,"history主题分析.pkl")

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\bidirectional
......vars
...layers\bidirectional\backward_layer
......vars
...layers\bidirectional\backward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\forward_layer
......vars
...layers\bidirectional\forward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\layer
......vars
...layers\bidirectional\layer\cell
......vars
...layers\conv2d
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\embedding
......vars
.........0
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\reshape
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
......

NameError: name 'history' is not defined